# Library Imports

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import io
# from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score,\
                            precision_score, recall_score, accuracy_score,\
                            average_precision_score, precision_recall_curve
from imblearn.over_sampling import RandomOverSampler
from src.helper_functions import load_data, get_model_perfs, init_model_perfs,\
                                 save_model, save_model_perfs, check_is_best,\
                                 read_model, evaluate_model_predictions,\
                                 update_model_perfs, check_and_save,\
                                 adjusted_classes
from src.metrics import recall_m, f1

Using TensorFlow backend.
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/sean/anaconda3/envs

In [2]:
import keras
from keras.callbacks import Callback
from keras.layers import Dense, Activation, Dropout
keras.__version__

'2.2.5'

# Load Data

In [3]:
x_tr, y_tr, x_te, y_te = load_data()

# Build Neural Network

Following the desciption in section 2.2.4 of the [DeepTox article](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full), I tried to use intermediate values in [Table 2](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full#T2) to build the neural network:

Following [this question/answer](https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model) and [this question/answer](https://stackoverflow.com/questions/54065733/how-to-employ-the-scikit-learn-evaluation-metrics-functions-with-keras-in-python) to implement usage of recall in model training:

In [4]:
drop_out=0.5    # DeepTox range: 0.5, 0.2, 0
L2_reg = 0.0001 # Default = 0.01
layers = 3      # DeepTox range: 1, 2, 3, 4
act = 'sigmoid' # Consider sigmoid and tanh
neurons = 1024  # DeepTox range: 1024, 2048, 4096, 8192, 16384
# Info on decay: https://datascience.stackexchange.com/questions/26112/decay-parameter-in-keras-optimizers
decay = 0       # DeepTox range: 10^-4, 10^-5, 10^-6
learn_rate = 0.1  #Research appropriate range
DNN = keras.Sequential([
    keras.layers.InputLayer(input_shape=x_tr.shape[1:],name='Input_Layer')
])
for i in range(1,layers+1):
    DNN.add(Dense(units=neurons, activation=act,\
                  name='h'+str(i)+'_'+act+'_activation',\
                  kernel_regularizer=keras.regularizers.l2(L2_reg)))
    DNN.add(Dropout(rate=drop_out,name='Dropout'+str(i)))
DNN.add(Dense(units=1, activation='sigmoid'))
keras.optimizers.Adam(lr=learn_rate, beta_1=0.9,\
                      beta_2=0.999, decay=decay, amsgrad=False)
DNN.compile(optimizer='adam', loss='binary_crossentropy',\
            metrics=['accuracy',recall_m,f1])
DNN.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
h1_sigmoid_activation (Dense (None, 1024)              1684480   
_________________________________________________________________
Dropout1 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h2_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout2 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h3_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________

# Train DNN's

In [5]:
for target in y_tr.columns[:1]:
    # Determine rows with available data
    rows_tr = np.isfinite(y_tr[target]).values
    rows_te = np.isfinite(y_te[target]).values
    x,y = x_tr[rows_tr], y_tr[target][rows_tr]
    
    # Address Class Imbalance
    x_train, x_val, y_train, y_val = train_test_split(x, y, stratify=y, \
                                                      test_size=0.2, random_state=42)
    ros = RandomOverSampler(random_state=0)
    x_resampled, y_resampled = ros.fit_sample(x_train,y_train)
    
    # Train the DNN
    DNN.fit(
        x_resampled, y_resampled, batch_size=512, epochs=100,\
        validation_data=(x_val,y_val), verbose=1,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=16,verbose=1,\
                                          restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)
        ])

Instructions for updating:
Use tf.cast instead.
Train on 11934 samples, validate on 1689 samples
Epoch 1/100
11934/11934 [==============================] - 3s 252us/step - loss: 1.1367 - acc: 0.5261 - recall_m: 0.5395 - f1: 0.4547 - val_loss: 0.8516 - val_acc: 0.5784 - val_recall_m: 0.7162 - val_f1: 0.2805
Epoch 2/100
11934/11934 [==============================] - 2s 193us/step - loss: 0.8714 - acc: 0.6030 - recall_m: 0.6011 - f1: 0.5946 - val_loss: 0.8097 - val_acc: 0.6495 - val_recall_m: 0.7696 - val_f1: 0.3348
Epoch 3/100
11934/11934 [==============================] - 3s 228us/step - loss: 0.7835 - acc: 0.6708 - recall_m: 0.7164 - f1: 0.6840 - val_loss: 0.6929 - val_acc: 0.7525 - val_recall_m: 0.7013 - val_f1: 0.3912
Epoch 4/100
11934/11934 [==============================] - 3s 219us/step - loss: 0.7524 - acc: 0.6880 - recall_m: 0.7181 - f1: 0.6951 - val_loss: 0.8868 - val_acc: 0.4796 - val_recall_m: 0.9629 - val_f1: 0.2991
Epoch 5/100
11934/11934 [==============================] - 

# Testing DNN Model Saving.

In [8]:
filename = save_model(target,'DNN', DNN, True)
filename

'DNN1.h5'

In [9]:
DNN1 = read_model(target,filename)

ValueError: You are trying to load a weight file containing 4 layers into a model with 0 layers

In [11]:
from joblib import dump, load
filename = dump(DNN, './models/NR_AhR/DNN1y.joblib')[0]
filename

'./models/NR_AhR/DNN1y.joblib'

In [12]:
DNN2 = load(filename)

ValueError: You are trying to load a weight file containing 4 layers into a model with 0 layers

In [13]:
import pickle
pickle.dump(DNN,open('./models/NR_AhR/DNN1z.pkl', 'wb'))

In [14]:
DNN3=pickle.load(open('./models/NR_AhR/DNN1z.pkl', 'rb'))

ValueError: You are trying to load a weight file containing 4 layers into a model with 0 layers

In [16]:
DNN = keras.Sequential()
DNN.add(Dense(neurons, activation=act,input_shape=x_tr.shape[1:],name='h0_'+act+'_activation'))
DNN.add(Dropout(rate=drop_out,name='Dropout0'))
for i in range(1,layers):
# DNN = keras.Sequential([
#     keras.layers.InputLayer(input_shape=x_tr.shape[1:],name='Input_Layer')
# ])
# for i in range(1,layers+1):
    DNN.add(Dense(units=neurons, activation=act,\
                  name='h'+str(i)+'_'+act+'_activation',\
                  kernel_regularizer=keras.regularizers.l2(L2_reg)))
    DNN.add(Dropout(rate=drop_out,name='Dropout'+str(i)))
DNN.add(Dense(units=1, activation='sigmoid'))
keras.optimizers.Adam(lr=learn_rate, beta_1=0.9,\
                      beta_2=0.999, decay=decay, amsgrad=False)
DNN.compile(optimizer='adam', loss='binary_crossentropy',\
            metrics=['accuracy',recall_m,f1])
DNN.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
h0_sigmoid_activation (Dense (None, 1024)              1684480   
_________________________________________________________________
Dropout0 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h1_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout1 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h2_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout2 (Dropout)           (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [17]:
for target in y_tr.columns[:1]:
    # Train the DNN
    DNN.fit(
        x_resampled, y_resampled, batch_size=512, epochs=100,\
        validation_data=(x_val,y_val), verbose=1,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=16,verbose=1,\
                                          restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)
        ])

Train on 11934 samples, validate on 1689 samples
Epoch 1/100
11934/11934 [==============================] - 3s 241us/step - loss: 1.1342 - acc: 0.5114 - recall_m: 0.5730 - f1: 0.4606 - val_loss: 0.7172 - val_acc: 0.8840 - val_recall_m: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 2/100
11934/11934 [==============================] - 2s 187us/step - loss: 0.8757 - acc: 0.5711 - recall_m: 0.5398 - f1: 0.5456 - val_loss: 0.7823 - val_acc: 0.5252 - val_recall_m: 0.8753 - val_f1: 0.2982
Epoch 3/100
11934/11934 [==============================] - 2s 185us/step - loss: 0.7829 - acc: 0.6523 - recall_m: 0.6846 - f1: 0.6622 - val_loss: 0.7420 - val_acc: 0.6821 - val_recall_m: 0.7851 - val_f1: 0.3628
Epoch 4/100
11934/11934 [==============================] - 2s 208us/step - loss: 0.7393 - acc: 0.6824 - recall_m: 0.7265 - f1: 0.6951 - val_loss: 0.6407 - val_acc: 0.7626 - val_recall_m: 0.7019 - val_f1: 0.4010
Epoch 5/100
11934/11934 [==============================] - 3s 217us/step - loss: 0.7169 - acc: 0.69

In [18]:
filename = save_model(target,'DNNtest', DNN, True)
filename

'DNNtest0.h5'

In [19]:
DNN1 = read_model(target,filename)

ValueError: Unknown metric function:recall_m

In [20]:
DNN = keras.Sequential()
DNN.add(Dense(neurons, activation=act,input_shape=x_tr.shape[1:],name='h0_'+act+'_activation'))
DNN.add(Dropout(rate=drop_out,name='Dropout0'))
for i in range(1,layers):
    DNN.add(Dense(units=neurons, activation=act,\
                  name='h'+str(i)+'_'+act+'_activation',\
                  kernel_regularizer=keras.regularizers.l2(L2_reg)))
    DNN.add(Dropout(rate=drop_out,name='Dropout'+str(i)))
DNN.add(Dense(units=1, activation='sigmoid'))
keras.optimizers.Adam(lr=learn_rate, beta_1=0.9,\
                      beta_2=0.999, decay=decay, amsgrad=False)
DNN.compile(optimizer='adam', loss='binary_crossentropy',\
            metrics=['accuracy'])
DNN.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
h0_sigmoid_activation (Dense (None, 1024)              1684480   
_________________________________________________________________
Dropout0 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h1_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout1 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h2_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout2 (Dropout)           (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [21]:
for target in y_tr.columns[:1]:
    # Train the DNN
    DNN.fit(
        x_resampled, y_resampled, batch_size=512, epochs=100,\
        validation_data=(x_val,y_val), verbose=1,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=16,verbose=1,\
                                          restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)
        ])

Train on 11934 samples, validate on 1689 samples
Epoch 1/100
11934/11934 [==============================] - 3s 252us/step - loss: 1.1021 - acc: 0.5243 - val_loss: 0.7184 - val_acc: 0.8437
Epoch 2/100
11934/11934 [==============================] - 2s 181us/step - loss: 0.8539 - acc: 0.5968 - val_loss: 0.7820 - val_acc: 0.5287
Epoch 3/100
11934/11934 [==============================] - 2s 183us/step - loss: 0.7715 - acc: 0.6656 - val_loss: 0.7733 - val_acc: 0.5554
Epoch 4/100
11934/11934 [==============================] - 2s 196us/step - loss: 0.7372 - acc: 0.6868 - val_loss: 0.7639 - val_acc: 0.5530

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/100
11934/11934 [==============================] - 2s 206us/step - loss: 0.7055 - acc: 0.7024 - val_loss: 0.6307 - val_acc: 0.7673
Epoch 6/100
11934/11934 [==============================] - 3s 213us/step - loss: 0.6786 - acc: 0.7206 - val_loss: 0.6475 - val_acc: 0.7519
Epoch 7/100
11934/11934 [===========

In [22]:
filename = save_model(target,'DNNtest', DNN, True)
filename

'DNNtest1.h5'

In [23]:
DNN1 = read_model(target,filename)

# THAT WORKS!
Now, verify that no input_layer and with custom metrics still fails!

In [24]:
DNN = keras.Sequential()
DNN.add(Dense(neurons, activation=act,input_shape=x_tr.shape[1:],name='h0_'+act+'_activation'))
DNN.add(Dropout(rate=drop_out,name='Dropout0'))
for i in range(1,layers):
    DNN.add(Dense(units=neurons, activation=act,\
                  name='h'+str(i)+'_'+act+'_activation',\
                  kernel_regularizer=keras.regularizers.l2(L2_reg)))
    DNN.add(Dropout(rate=drop_out,name='Dropout'+str(i)))
DNN.add(Dense(units=1, activation='sigmoid'))
keras.optimizers.Adam(lr=learn_rate, beta_1=0.9,\
                      beta_2=0.999, decay=decay, amsgrad=False)
DNN.compile(optimizer='adam', loss='binary_crossentropy',\
            metrics=['accuracy',recall_m,f1])
DNN.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
h0_sigmoid_activation (Dense (None, 1024)              1684480   
_________________________________________________________________
Dropout0 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h1_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout1 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h2_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout2 (Dropout)           (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                

In [25]:
for target in y_tr.columns[:1]:
    # Train the DNN
    DNN.fit(
        x_resampled, y_resampled, batch_size=512, epochs=100,\
        validation_data=(x_val,y_val), verbose=1,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=16,verbose=1,\
                                          restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)
        ])

Train on 11934 samples, validate on 1689 samples
Epoch 1/100
11934/11934 [==============================] - 3s 259us/step - loss: 0.9880 - acc: 0.5466 - recall_m: 0.5765 - f1: 0.5074 - val_loss: 0.7037 - val_acc: 0.7602 - val_recall_m: 0.5331 - val_f1: 0.3339
Epoch 2/100
11934/11934 [==============================] - 2s 179us/step - loss: 0.8081 - acc: 0.6449 - recall_m: 0.6655 - f1: 0.6474 - val_loss: 0.7440 - val_acc: 0.7010 - val_recall_m: 0.7410 - val_f1: 0.3625
Epoch 3/100
11934/11934 [==============================] - 2s 179us/step - loss: 0.7499 - acc: 0.6832 - recall_m: 0.7272 - f1: 0.6946 - val_loss: 0.6202 - val_acc: 0.7916 - val_recall_m: 0.6557 - val_f1: 0.4168
Epoch 4/100
11934/11934 [==============================] - 3s 210us/step - loss: 0.7179 - acc: 0.6988 - recall_m: 0.7311 - f1: 0.7070 - val_loss: 0.7024 - val_acc: 0.6240 - val_recall_m: 0.8482 - val_f1: 0.3425
Epoch 5/100
11934/11934 [==============================] - 2s 200us/step - loss: 0.6956 - acc: 0.7049 - rec

Epoch 37/100
11934/11934 [==============================] - 2s 203us/step - loss: 0.5003 - acc: 0.7941 - recall_m: 0.8102 - f1: 0.7971 - val_loss: 0.4933 - val_acc: 0.8200 - val_recall_m: 0.7303 - val_f1: 0.4786
Epoch 38/100
11934/11934 [==============================] - 2s 205us/step - loss: 0.4983 - acc: 0.7953 - recall_m: 0.8129 - f1: 0.7987 - val_loss: 0.4933 - val_acc: 0.8218 - val_recall_m: 0.7303 - val_f1: 0.4809

Epoch 00038: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 39/100
11934/11934 [==============================] - 2s 205us/step - loss: 0.4990 - acc: 0.7959 - recall_m: 0.8098 - f1: 0.7981 - val_loss: 0.4920 - val_acc: 0.8206 - val_recall_m: 0.7303 - val_f1: 0.4792
Restoring model weights from the end of the best epoch
Epoch 00039: early stopping


In [26]:
filename = save_model(target,'DNNtest', DNN, True)
filename

'DNNtest2.h5'

In [27]:
DNN1 = read_model(target,filename)

ValueError: Unknown metric function:recall_m